# Tweakers ISOP
This file contains a numerical approach to solve the internet shopping optimization problem (ISOP). The idea of the problem is that a number of items need to be purchased and that per item, there are a number of stores that offer a price for the item. Aside from this, each store may also have shipping costs, which are imposed once for all items. 

This script solves this using a Q-learning based approach, where instead of learning the quality of each store, this is determined using the expected costs of each store and by regarding the number of items sold by the store. 






In [292]:
##Config

#Paste here the links from tweakers. Make sure that for each link, you have selected the prices page and not the specifications page.
links = [
    "https://tweakers.net/pricewatch/1856828/amd-ryzen-9-7900x-boxed.html",
    "https://tweakers.net/pricewatch/1800156/kingston-fury-beast-kf556c40bbk2-64.html",
    "https://tweakers.net/pricewatch/1648262/samsung-980-pro-zonder-heatsink-2tb.html",
    "https://tweakers.net/pricewatch/1699078/corsair-rm1000x-2021-zwart.html",
    "https://tweakers.net/pricewatch/1823552/nzxt-h7-flow-zwart-wit.html",
    "https://tweakers.net/pricewatch/1472978/noctua-nh-d15-chromax-punt-black.html",
    "https://tweakers.net/pricewatch/1274405/arctic-p14-pwm-pst-value-pack-5-stuks-140mm.html",
    "https://tweakers.net/pricewatch/1870844/asus-tuf-gaming-x670e-plus-wifi.html"   
]

#Paste here names of stores you want to exclude during analysis
blacklist= ["RedShell.nl | zakelijk"] 



#Tweakers does often not accurately track the shipping costs of companies. Therefore, you can override the shipping costs here
shipping_cost_companies = ["Proshop.nl","SWW Computer"]
shipping_cost_override = [6.99,6.95]

override = pd.DataFrame()
override['Store'] = shipping_cost_companies
override['Shipping'] = shipping_cost_override

Run the below cell to fetch the prices for your items

In [306]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import numpy as np
def fetch_cheapest_entry(link, blacklist=[]):
    #print(link)
    headers= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
    response = requests.get(link, headers)
    if response.status_code != 200:
        #ValueError("Error fetching page")
        print("Error fetching page, please wait a while. This is probably because of the bot-blocker from Tweakers")
        return None
    else:
        content = response.content
    # print(content)


    df = pd.DataFrame()
    soup = BeautifulSoup(response.content, 'html.parser')
    title=soup.find('h1').getText()
    if title == "Tweakers" or title=="None" or title==None or title=="":
        return None
    print("Fetched:" + str(title))
    athing = soup.find_all(class_="shop-listing")
    #print(athing)
    #return
    i=0;
    for a in athing: #Find shop listing
        i+=1
        if i==len(athing):
            links=[]
            for q in ['shop-name','shop-bare-price','shop-price']:
                entries=[]
                for b in a.find_all(class_=q):
                    if q == 'shop-name':
                        links.append(b.find('a')['href'])
                        #print(b.find('a')['href'])
                    string=b.find('a').getText().strip()
                    string=string.replace(u'€\xa0', u'')
                    if q != 'shop-name':
                        string=string.replace(u'-', u'00')
                    string=string.replace(u',', u'.')
                    entries.append(string)
                df[q]=entries

            df['link'] = links
    df['title']=np.repeat(title,len(entries))
    for name in blacklist:
        df = df.drop(df[df['shop-name']==name].index) #remove blacklisted shops
        #df = df.head(3)
    
    
    
    for name in df['shop-name']:
        if name in shipping_cost_companies:
            #print(df[df['shop-name']==name]['shop-bare-price'])
            df.loc[df['shop-name']==name,'shop-price'] = float(df[df['shop-name']==name]['shop-bare-price']) + float(override[override['Store']==name]['Shipping'])# = float(list(df[df['shop-name']==name]['shop-bare-price'])[0]) + float(list(override[override['Store']==name]['Shipping'])[0])
            #df[df['shop-name']==name]['shop-price'] = df[df['shop-name']==name]['shop-bare-price'] + override[override['Store']==name]['Shipping']
    return(df)
        #shop-price

        
#fetch_cheapest_entry("menneus")


df = pd.DataFrame()

#products = ["7900x", "Kingston Fury 5600 64GB", "Samsung 980 PRO 2TB","x670e plus","RM 1000x", "H7 flow", "NH-D15 zwart", "P14 PWM PST valuepack"]
p=0;
l=[]
print("Fetching prices, please wait...")
for link in links:
    df=None
    t=1
    while(type(df)==type(None)):
        df=fetch_cheapest_entry(link,blacklist)
        if ((type(df)!=type(None))):
            t=1
        time.sleep(t)
        t+=random.randint(3,5)
    #print(df)
    l.append(df)
    #display(df)
    
    p+=1
print("Done fetching prices.")

Fetching prices, please wait...
Fetched:AMD Ryzen 9 7900X Boxed 
Fetched:Kingston Fury Beast KF556C40BBK2-64 
Fetched:Samsung 980 Pro (zonder heatsink) 2TB 
Fetched:Corsair RM1000x (2021) Zwart 
Fetched:NZXT H7 Flow Zwart/Wit 
Error fetching page, please wait a while. This is probably because of the bot-blocker from Tweakers
Error fetching page, please wait a while. This is probably because of the bot-blocker from Tweakers
Error fetching page, please wait a while. This is probably because of the bot-blocker from Tweakers
Error fetching page, please wait a while. This is probably because of the bot-blocker from Tweakers
Fetched:Noctua NH-D15 chromax.black 
Fetched:Arctic P14 PWM PST Value Pack (5 stuks), 140mm 
Fetched:ASUS TUF Gaming X670E-PLUS WIFI 
Done fetching prices.


Run the cell below to pre-process the data

In [307]:

from collections import Counter
minindices=np.zeros(len(l))
shops = set([]);


n = len(l)#number of products
for i in range(len(l)):
    for name in l[i]['shop-name']:
        shops.add(name)

m=len(shops)
#print(m)


prices = np.zeros((n, m))
delivery_costs = np.zeros((n,m))
for i in range(len(l)): #for each product
    for j in range(len(shops)):
        price= l[i][l[i]['shop-name']==list(shops)[j]]['shop-bare-price']
        price_with_delivery = l[i][l[i]['shop-name']==list(shops)[j]]['shop-price']
        if len(price)!=0:
            delivery_cost = float(price_with_delivery) - float(price)
            prices[i,j] = float(price)
            delivery_costs[i,j] = delivery_cost
        else:
            prices[i,j]= np.inf
            delivery_costs[i,j] = np.inf
            
#     n: the number of products
#     m: the number of stores
#     prices: an n-by-m numpy array of the prices for each product at each store
#     delivery_costs: an n-by-m numpy array of the delivery costs for each product at each store


#display(pd.DataFrame(delivery_costs))

Run the below cell to find the optimal purchasing policy

In [310]:
max_stores = n
def calculate_price_from_indexlist(choices,possible_shops):
    cost = 0;
    shipping=0;
    i=0;
    for store in choices:
        cost += prices[i,int(store)]
        i+=1;
    for store in possible_shops:
        shipping+= pd.DataFrame(delivery_costs).iloc[:,store].replace(np.inf,np.nan).dropna().median()
    cost+=shipping
    return cost, shipping


a=np.array(list(shops))

qa = np.zeros(m)
for i in range(n):
    min_cost=np.inf
    for j in range(m):
        if prices[i,j] != np.inf:
            qa[j]+=np.mean(prices[i][prices[i]!=np.inf])-prices[i,j]#np.min(prices[i][prices[i]!=0]);
            #Qa is a quality metric that encodes an average price saving.           
qa[qa<0]=0 #We remove the options that are on average more expensive. 
qa=qa/sum(qa)

#for j in range(m):
bin_stock = prices.copy();
bin_stock[bin_stock == np.inf] = 0
bin_stock = bin_stock>0
bin_stock = pd.DataFrame(bin_stock)
qb=np.array(bin_stock.sum(axis=0))
qb = qb/sum(qb) #qb encodes how many products can be bought at a given store. More products means higher quality.


qc = (qa+qb)/sum(qa+qb) #Our total quality metric is the avereage of both previous metrics


#qb
 #qa
    
from numpy.random import choice
min_cost=np.inf;
min_shipping=0;
optimal_stores=0;
for k in range(20000):
    nstores = np.random.randint(max_stores)+1  
    valid = False
    while(not (valid)): #this loop makes sure that all products are available at the selected stores.
        stores = choice(range(m), nstores, p=qc, replace=False)
        if sum(np.array(bin_stock[stores].sum(axis=1))>0) == n:
            valid=True



    choices = np.zeros(n)        
    for i in range(n):
        choices[i] = stores[prices[i,stores].argmin()]

    storez = list(set(stores) - (set(stores) - set(choices))) #Remove the stores where we did not choose to order anything

    #print(choices)
    cost, shipping = calculate_price_from_indexlist(choices,storez)
    
    if cost<min_cost:
        
        min_cost=cost;
        min_shipping=shipping;
        max_q=0;
        optimal_stores = choices
        for f in list(set(choices)):
            max_q+=qc[int(f)]
        print("Current cheapest cost: "+ str(round(cost*100)/100) + "with Q value of " + str(round(max_q*1000)/1000)) 
# print()
print("Done! Here is the optimal purchasing policy:")
print(optimal_stores, min_cost, min_shipping)

sa = []
sb= []
sc=[]
sd = []
for a,i in zip(optimal_stores,range(n)):
    sa.append(l[i]['title'][0])
    sb.append(list(shops)[int(a)])
    sc.append(list(l[i][l[i]['shop-name']==list(shops)[int(a)]]['link'])[0])
    sd.append(list(l[i][l[i]['shop-name']==list(shops)[int(a)]]['shop-bare-price'])[0])


Current cheapest cost: 1680.75with Q value of 0.178
Current cheapest cost: 1679.92with Q value of 0.221
Current cheapest cost: 1674.78with Q value of 0.331
Current cheapest cost: 1671.03with Q value of 0.245
Current cheapest cost: 1660.0with Q value of 0.384
Current cheapest cost: 1659.22with Q value of 0.324
Current cheapest cost: 1656.97with Q value of 0.171
Current cheapest cost: 1653.93with Q value of 0.26
Current cheapest cost: 1653.77with Q value of 0.306
Current cheapest cost: 1651.87with Q value of 0.243
Done! Here is the optimal purchasing policy:
[ 2. 18. 18. 21. 24. 26. 18. 21.] 1651.8700000000001 17.890000000000004


In [311]:
df = pd.DataFrame()
df['Product'] = sa
df['Store'] = sb
df['Link'] = sc
df['Price'] = sd

#print(sc)

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, "link")

df2=df.style.format({'Link': make_clickable})
print("Purchasing policy:")
display(df2)

tp = []
for j in range(m):
#    if j == optimal_store
    if j in optimal_stores:
        sf = pd.DataFrame()
        shopname=list(shops)[int(j)]
        print("--------------------------------")
        print("Shopping cart for: " + str(shopname))
        sf['Product'] = list(df[df['Store'] == shopname]['Product'])
        sf['Link'] = list(df[df['Store'] == shopname]['Link'])
        sf['Price'] = list(df[df['Store'] == shopname]['Price'])
        sf2=sf.style.format({'Link': make_clickable})
        display(sf2)


        total_price = pd.to_numeric(sf['Price']).sum()

        gg=pd.DataFrame()
        gg["s"] = list(shops)
        idx=list(gg.index[gg['s']==shopname])[0]
        dil=pd.DataFrame(delivery_costs).min(axis=0)[idx]
        jst =dil+total_price
        tp.append(jst)
        print(str(total_price) + " + " + str(round(dil*100)/100) + " = "+ str(jst))  
        print("--------------------------------")

print("So, total cost of all purchases combined is: " + str(sum(tp)))

Purchasing policy:


,Product,Store,Link,Price
0,AMD Ryzen 9 7900X Boxed,Alternate.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=156&pe-id=1856828&link=2cb270f42979bbe40b8c977ab0b6c69cc2b52d7aa52eb7a1834443db254521b27376a784ba24242558408374756049e5027f9e58e92f25fae556856f2c362800e5b122843d0cdac6e5ab4ffe06069aabbc4551ee3713984b9f0eb806fd095b63371376d68148e6186740d253c04f082e936ac503782780b0c0a4b1e05e0062b95e01dcd5ee69c05457b5b9719b297c403347c5b9cae9dc51883f71cb64af2a0b97f4bf28c6354f7fb4b8a505d905cf2714cd30bd703c6d227a2e71ac6316d1812630021c7dbdf9b069e4b8eb7e80188553345fd801&nonce=04a2d05f71f16082c03e7d6bf56c05aa79e6871f1f554a99"">link",419.00
1,Kingston Fury Beast KF556C40BBK2-64,Proshop.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=10990&pe-id=1800156&link=0932c42e4606a680e92bb68725f1c109c0544a61cedfc43c34a3c5e6b5cbd17851b055d188e82d65410a810e91285778631aef0de9e088fa715b33e8c3b8ad044c3fc19951f192823b26793ba256aa52e62c11527cf288e9587555003eb037cc8534c5dc8b48b57e7bbf1bb1136624031a034f09f9450fa2bbe234502656117268a0c872a7ed3680dae22d4f53331ade59b157803ce001cabd62ee9ad62bc38d595788b2ab7b79eb26e7ee369c9f2aabc1674a95991656b36708bd6ac71211a246bc3265b3700cf6e894ed7754f63aa86f0fc3deef1e4cf8f1a250ccf6&nonce=1400096b6f8e18f127b43411daa5f38c3b0b7dfce2b28ac7"">link",245.96
2,Samsung 980 Pro (zonder heatsink) 2TB,Proshop.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=10990&pe-id=1648262&link=664be9597c7240f6c7c0bca8d7ede4a0489d8984b5deb572ecc64688bc76c8c83077d5f2d2b9f897394bfcbcab5d6f551a3a63c8c5cea2e5efde0b96747b033ea938aeb06093828fa8ef4adccb0e5dd6f69677a2bae301e075b47203caaf483b371038fc800e7de2a6cf65c00d31637e89978494c87fd7c56133e40e29f61ec0245e0596767b0c817ec213df98eb0b491216f4dbd101d83c4e9782b5dd3c7764bc293b613a683ad8ecbf5ba8c9e2078c5ca02de76d7121fd026adf00cec80e75d2a83af8b93e88ade8009e521c12e6365f5fe7256a3e02b61b4ba5&nonce=594df7e249fda40af300a0cc05e464a8844ffea087331ccc"">link",171.97
3,Corsair RM1000x (2021) Zwart,Amazon.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=10654&pe-id=1699078&link=131c439c35d0f61e8ee260c8fde773a6c19e23aba29f6a07922a1026e150c4c313a09371936de7b510df3e8b4989641dd5165f656b5ecb8c019a7866f10516b8f55bb55c88a229ff097b1689e3a35a16f48fa6f6ca7da1c1f498ade11ad86cd1300c25d83e4fc7c177472e244cdde58e9f2b2c159d91c06fd44a1d5479b2041cbd03b7bd82b40460aa4aff4fab93da7db382e655474d529de0a3d327fb6916d74837c154d20bfbae4ca7aa7c0e4d98abe0a1866694c286931687cbeb195075698b0c36684aa3fbfe149a0b9bcdd7596119b9f750bf7b16855777148d2ec95266924a34eafb&nonce=dbebb6bbf4e4b084ceea86a4bceb800955cdf9e611f20a54"">link",194.50
4,NZXT H7 Flow Zwart/Wit,HardwareWebwinkel.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=6743&pe-id=1823552&link=831ba74df49fa745addedeed7b3a033a9dc7acde96c5246823fc783784c0c75b04fa0b2b7b93923162420b2e408efd21de8077bee4065ccf6c0bad7e3f4f86c46e76f984c97b678c8e2b0927350971a6d75094a5f3b7f47234c121d4384a4910c6d5697ddcc55f663a8e4814a6b44ea1d46d5e293c0986577d4c4eb390bb028a007fbb50c8c96397130175b9b6907d6980565cafd9bb71a2649a1c8c578cc826741ab80317e511d583060d6af0c47f08f279a2d40642b3b46478276b3e35e071cd25202501d706e9fc4abd495a0df0bb33f17a74a9979627d37937bb1fd812be65d6c0f62fdb0b9c382bf896639c4e1db78ce5c986c77ff2b1fe0fd4b3de035eb2cae2f1f8e17f036137852c9e5def6846ed8cf5f2f536b488228535956ab16997576af7b27dfd447bd42b0ab63a9884623aa9692f7f212b912358cc1fa88005410ea71dfaa996e4de5b8510dbd94bc0357d07b1ca43309272a30a199bb3b459987958f9c0a5b9e6b7d05c0610826de3a70dee1cfbadbc63563166a70682fa28393ea3&nonce=4e2bafdc51ff396b97b960864ca752aedb753ef526d34641"">link",123.10
5,Noctua NH-D15 chromax.black,Hardware-Experts,link,114.95
6,"Arctic P14 PWM PST Value Pack (5 stuks), 140mm",Proshop.nl,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=10990&pe-id=1274405&link=070acdf578839e30e041642dcd9120c3d9d10a58535d9a0e95e95abf0cadec1b8ea6e580bb6b888d931e6c1c73ca6ccda1246f681d830ad703e7e4c

--------------------------------
Shopping cart for: Alternate.nl


,Product,Link,Price
0,AMD Ryzen 9 7900X Boxed,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=156&pe-id=1856828&link=2cb270f42979bbe40b8c977ab0b6c69cc2b52d7aa52eb7a1834443db254521b27376a784ba24242558408374756049e5027f9e58e92f25fae556856f2c362800e5b122843d0cdac6e5ab4ffe06069aabbc4551ee3713984b9f0eb806fd095b63371376d68148e6186740d253c04f082e936ac503782780b0c0a4b1e05e0062b95e01dcd5ee69c05457b5b9719b297c403347c5b9cae9dc51883f71cb64af2a0b97f4bf28c6354f7fb4b8a505d905cf2714cd30bd703c6d227a2e71ac6316d1812630021c7dbdf9b069e4b8eb7e80188553345fd801&nonce=04a2d05f71f16082c03e7d6bf56c05aa79e6871f1f554a99"">link",419.00


419.0 + 5.95 = 424.95
--------------------------------
--------------------------------
Shopping cart for: Proshop.nl


...

523.5 + 0.0 = 523.5
--------------------------------
--------------------------------
Shopping cart for: HardwareWebwinkel.nl


,Product,Link,Price
0,NZXT H7 Flow Zwart/Wit,"<a target=""_blank"" href=""https://tweakers.net/pricewatch/clickout?shop-id=6743&pe-id=1823552&link=831ba74df49fa745addedeed7b3a033a9dc7acde96c5246823fc783784c0c75b04fa0b2b7b93923162420b2e408efd21de8077bee4065ccf6c0bad7e3f4f86c46e76f984c97b678c8e2b0927350971a6d75094a5f3b7f47234c121d4384a4910c6d5697ddcc55f663a8e4814a6b44ea1d46d5e293c0986577d4c4eb390bb028a007fbb50c8c96397130175b9b6907d6980565cafd9bb71a2649a1c8c578cc826741ab80317e511d583060d6af0c47f08f279a2d40642b3b46478276b3e35e071cd25202501d706e9fc4abd495a0df0bb33f17a74a9979627d37937bb1fd812be65d6c0f62fdb0b9c382bf896639c4e1db78ce5c986c77ff2b1fe0fd4b3de035eb2cae2f1f8e17f036137852c9e5def6846ed8cf5f2f536b488228535956ab16997576af7b27dfd447bd42b0ab63a9884623aa9692f7f212b912358cc1fa88005410ea71dfaa996e4de5b8510dbd94bc0357d07b1ca43309272a30a199bb3b459987958f9c0a5b9e6b7d05c0610826de3a70dee1cfbadbc63563166a70682fa28393ea3&nonce=4e2bafdc51ff396b97b960864ca752aedb753ef526d34641"">link",123.10


123.1 + 4.95 = 128.04999999999998
--------------------------------
--------------------------------
Shopping cart for: Hardware-Experts


,Product,Link,Price
0,Noctua NH-D15 chromax.black,link,114.95


114.95 + 0.0 = 114.95
--------------------------------
So, total cost of all purchases combined is: 1651.87
